In [1]:
import requests
from IPython.display import display
import pandas as pd
from pathlib import Path
import xml.etree.ElementTree as ET
import re
import unicodedata
import openpyxl as xl
from openpyxl.worksheet.worksheet import Worksheet
from openpyxl.cell.cell import MergedCell
from typing import List, Tuple
import json

In [2]:
out_dir: Path = Path("../out")
if not out_dir.exists():
    out_dir.mkdir(parents=True, exist_ok=True)

data_dir: Path = Path("../public/data")
if not data_dir.exists():
    data_dir.mkdir(parents=True, exist_ok=True)

In [3]:
orig_url = "https://www.digital.go.jp/assets/contents/node/basic_page/field_ref_resources/c43e8643-e807-41f3-b929-94fb7054377e/b475efa7/20221221_meeting_administrative_research_outline_11.xlsx"
orig_xlsx: Path = out_dir / orig_url.split("/")[-1]

def download_file(url, dest):
    b = requests.get(url).content
    with open(dest, mode="wb") as f:
        f.write(b)

if not orig_xlsx.exists():
    download_file(orig_url, orig_xlsx)

In [4]:
orig = (
    pd.read_excel(
        orig_xlsx,
        sheet_name="工程表一覧表",
        skiprows=1,
        skipfooter=24,
        dtype=str,
    )
    .rename(
        columns=(lambda s: s.replace("\n", "").replace(" ", "").strip()),
    )
    .rename(
        columns=(lambda s: "見直し要否" if s.startswith("見直し要否") else s),
    )
)
display(orig)

工程表一覧表
工程表の類型


,分類,No.,法令名,所管省庁名,条項,規制等の内容概要,規制等の類型,現在Phase,見直後Phase,見直し要否,見直し完了時期,工程表,見直しの概要
0,別表１,1,人事記録の記載事項等に関する政令,内閣官房,第5条,検査,目視規制,1-①,2,要,令和４年度１月～３月,目視ー共通１,告示、通知・通達等の発出又は改正
1,別表１,3,人事院規則13―３（災害補償の実施に関する審査の申立て等）,人事院,第31条,災害補償審査委員会が必要と認める場合の実地調査,目視規制,1-①,2,要,令和４年度１月～３月,目視ー共通１,告示、通知・通達等の発出又は改正
2,別表１,4,公文書等の管理に関する法律,内閣府,第9条第3項,管理状況の報告等,目視規制,1-①,2,要,完了済み,NaN,告示、通知・通達等の発出又は改正
3,別表１,5,公文書等の管理に関する法律,内閣府,第9条第4項,管理状況の報告等,目視規制,1-①,2,要,完了済み,NaN,告示、通知・通達等の発出又は改正
4,別表１,6,国家戦略特別区域法,内閣府,第12条の3第9項,学校教育法等の特例,目視規制,1-①,2,要,令和４年度１月～３月,目視ー共通１,告示、通知・通達等の発出又は改正
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7527,新規,435,商品先物取引法,経済産業省\n農林水産省,第144条の21第3項,新設合併契約に関する書面の閲覧,往訪閲覧,2-3①,3-3,要,令和５年度４月～９月,閲覧縦覧ー共通１,告示、通知・通達等の発出又は改正
7528,新規,436,商品先物取引法,経済産業省\n農林水産省,第302条第3項,委託者保護基金への資料の閲覧,往訪閲覧,2-3①,3-3,要,令和５年度４月～９月,閲覧縦覧ー共通１,告示、通知・通達等の発出又は改正
7529,新規,437,商品投資に係る事業の規制に関する法律,経済産業省\n農林水産省,第24条第1項,商品投資顧問契約に係る顧客の財産に関する帳簿書類の閲覧,往訪閲覧,2-3①,3-3,要,令和５年度４月～９月,閲覧縦覧ー共通１,告示、通知・通達等の発出又は改正
7530,新規,438,海洋汚染等及び海上災害の防止に関する法律,国土交通省,第９条の14,財務諸表等の閲覧等,往訪閲覧,2-3①\n2-3②\n2-3③,3-3,要,令和５年度10月～３月,閲覧縦覧ー共通５,告示、通知・通達等の発出又は改正


In [5]:
lawlist_url = "https://elaws.e-gov.go.jp/api/1/lawlists/1"
lawlist_xml = requests.get(lawlist_url).text
lawlist_tree = ET.fromstring(lawlist_xml)
lawinfos = [
    {
        "LawId": info.find("LawId").text or "",
        "LawTitle": re.sub(r"\s+抄$", "", (info.find("LawName").text or "")),
        "LawNum": info.find("LawNo").text or "",
    }
    for info in lawlist_tree.find("ApplData").iter("LawNameListInfo")
]
lawinfo_by_title = {
    info["LawTitle"]: info
    for info in lawinfos
}
print(f"{len(lawinfos) = }")

len(lawinfos) = 8710


In [6]:
def lawtitle_to_info(lawtitle: str):
    lawtitle = lawtitle.strip()
    title_kanjinum = lawtitle.replace("(", "（").replace(")", "）")
    info = lawinfo_by_title.get(lawtitle.strip(), None)
    if info:
        return info
    else:
        title_with_parens = f"（{lawtitle.strip()}）"

        for title in lawinfo_by_title:
            if title_with_parens in title:
                info = lawinfo_by_title[title]
                return info
            
        title_kanjinum = re.sub(r"\d", lambda s: "〇一二三四五六七八九"[int(s[0])], lawtitle)
        info = lawinfo_by_title.get(title_kanjinum, None)
            
        return info


def assign_info(df: pd.DataFrame):
    infos_df = pd.json_normalize(df["法令名"].apply(lawtitle_to_info).fillna({}))
    df = pd.concat([df, infos_df], axis=1)
    return df

df_with_info = assign_info(orig)
display(df_with_info)
(
    df_with_info
    .pipe(lambda df: df[df["LawNum"].isna()])
    .to_excel(out_dir / "unrecognized_lawtitle.xlsx")
)

,分類,No.,法令名,所管省庁名,条項,規制等の内容概要,規制等の類型,現在Phase,見直後Phase,見直し要否,見直し完了時期,工程表,見直しの概要,LawId,LawTitle,LawNum
0,別表１,1,人事記録の記載事項等に関する政令,内閣官房,第5条,検査,目視規制,1-①,2,要,令和４年度１月～３月,目視ー共通１,告示、通知・通達等の発出又は改正,341CO0000000011,人事記録の記載事項等に関する政令,昭和四十一年政令第十一号
1,別表１,3,人事院規則13―３（災害補償の実施に関する審査の申立て等）,人事院,第31条,災害補償審査委員会が必要と認める場合の実地調査,目視規制,1-①,2,要,令和４年度１月～３月,目視ー共通１,告示、通知・通達等の発出又は改正,351RJNJ13003000,人事院規則一三―三（災害補償の実施に関する審査の申立て等）,昭和五十一年人事院規則一三―三
2,別表１,4,公文書等の管理に関する法律,内閣府,第9条第3項,管理状況の報告等,目視規制,1-①,2,要,完了済み,NaN,告示、通知・通達等の発出又は改正,421AC0000000066,公文書等の管理に関する法律,平成二十一年法律第六十六号
3,別表１,5,公文書等の管理に関する法律,内閣府,第9条第4項,管理状況の報告等,目視規制,1-①,2,要,完了済み,NaN,告示、通知・通達等の発出又は改正,421AC0000000066,公文書等の管理に関する法律,平成二十一年法律第六十六号
4,別表１,6,国家戦略特別区域法,内閣府,第12条の3第9項,学校教育法等の特例,目視規制,1-①,2,要,令和４年度１月～３月,目視ー共通１,告示、通知・通達等の発出又は改正,425AC0000000107,国家戦略特別区域法,平成二十五年法律第百七号
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7527,新規,435,商品先物取引法,経済産業省\n農林水産省,第144条の21第3項,新設合併契約に関する書面の閲覧,往訪閲覧,2-3①,3-3,要,令和５年度４月～９月,閲覧縦覧ー共通１,告示、通知・通達等の発出又は改正,325AC0000000239,商品先物取引法,昭和二十五年法律第二百三十九号
7528,新規,436,商品先物取引法,経済産業省\n農林水産省,第302条第3項,委託者保護基金への資料の閲覧,往訪閲覧,2-3①,3-3,要,令和５年度４月～９月,閲覧縦覧ー共通１,告示、通知・通達等の発出又は改正,325AC0000000239,商品先物取引法,昭和二十五年法律第二百三十九号
7529,新規,437,商品投資に係る事業の規制に関する法律,経済産業省\n農林水産省,第24条第1項,商品投資顧問契約に係る顧客の財産に関する帳簿書類の閲覧,往訪閲覧,2-3①,3-3,要,令和５年度４月～９月,閲覧縦覧ー共通１,告示、通知・通達等の発出又は改正,403AC0000000066,商品投資に係る事業の規制に関する法律,平成三年法律第六十六号
7530,新規,438,海洋汚染等及び海上災害の防止に関する法律,国土交通省,第９条の14,財務諸表等の閲覧等,往訪閲覧,2-3①\n2-3②\n2-3③,3-3,要,令和５年度10月～３月,閲覧縦覧ー共通５,告示、通知・通達等の発出又は改正,345AC0000000136,海洋汚染等及び海上災害の防止に関する法律,昭和四十五年法律第百三十六号


In [7]:
def transform_num(s: str):
    return unicodedata.normalize("NFKC", "_".join([ss for ss in re.split(r"\D+", s) if ss]))

ptn_clause_num = re.compile(r"(?P<sp>附則)?(?P<a>第\d+条(?:の\d+)*)?(?P<p>第\d+項)?(?P<i>第\d+号(?:の\d+)*)?")
ptn_appdx_table_num = re.compile(r"別表(?P<appdx_table>第?\d+号?(?:の\d+)*)")
def parse_clause_num(s: str):
    s = re.sub(r"\s+", "", s)
    m = ptn_clause_num.match(s)
    if m and m[0]:
        return {
            "sp": m["sp"] and "sp",
            "a": m["a"] and transform_num(m["a"]),
            "p": m["p"] and transform_num(m["p"]),
            "i": m["i"] and transform_num(m["i"]),
        }
    
    m = ptn_appdx_table_num.match(s)
    if m:
        return {
            "AppdxTable": m["appdx_table"] and transform_num(m["appdx_table"]),
        }

def assign_clause_num(df: pd.DataFrame):
    clause_num_df = pd.json_normalize(df["条項"].apply(parse_clause_num).fillna({}))
    df = pd.concat([df, clause_num_df], axis=1)
    return df

df_with_clause_num = assign_clause_num(df_with_info)
display(df_with_clause_num)
(
    df_with_clause_num
    .pipe(lambda df: df[df[["sp", "a", "p", "i", "AppdxTable"]].isna().all(axis="columns")])
    .to_excel(out_dir / "unrecognized_clause_num.xlsx")
)

,分類,No.,法令名,所管省庁名,条項,規制等の内容概要,規制等の類型,現在Phase,見直後Phase,見直し要否,...,工程表,見直しの概要,LawId,LawTitle,LawNum,sp,a,p,i,AppdxTable
0,別表１,1,人事記録の記載事項等に関する政令,内閣官房,第5条,検査,目視規制,1-①,2,要,...,目視ー共通１,告示、通知・通達等の発出又は改正,341CO0000000011,人事記録の記載事項等に関する政令,昭和四十一年政令第十一号,None,5,None,None,NaN
1,別表１,3,人事院規則13―３（災害補償の実施に関する審査の申立て等）,人事院,第31条,災害補償審査委員会が必要と認める場合の実地調査,目視規制,1-①,2,要,...,目視ー共通１,告示、通知・通達等の発出又は改正,351RJNJ13003000,人事院規則一三―三（災害補償の実施に関する審査の申立て等）,昭和五十一年人事院規則一三―三,None,31,None,None,NaN
2,別表１,4,公文書等の管理に関する法律,内閣府,第9条第3項,管理状況の報告等,目視規制,1-①,2,要,...,NaN,告示、通知・通達等の発出又は改正,421AC0000000066,公文書等の管理に関する法律,平成二十一年法律第六十六号,None,9,3,None,NaN
3,別表１,5,公文書等の管理に関する法律,内閣府,第9条第4項,管理状況の報告等,目視規制,1-①,2,要,...,NaN,告示、通知・通達等の発出又は改正,421AC0000000066,公文書等の管理に関する法律,平成二十一年法律第六十六号,None,9,4,None,NaN
4,別表１,6,国家戦略特別区域法,内閣府,第12条の3第9項,学校教育法等の特例,目視規制,1-①,2,要,...,目視ー共通１,告示、通知・通達等の発出又は改正,425AC0000000107,国家戦略特別区域法,平成二十五年法律第百七号,None,12_3,9,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7527,新規,435,商品先物取引法,経済産業省\n農林水産省,第144条の21第3項,新設合併契約に関する書面の閲覧,往訪閲覧,2-3①,3-3,要,...,閲覧縦覧ー共通１,告示、通知・通達等の発出又は改正,325AC0000000239,商品先物取引法,昭和二十五年法律第二百三十九号,None,144_21,3,None,NaN
7528,新規,436,商品先物取引法,経済産業省\n農林水産省,第302条第3項,委託者保護基金への資料の閲覧,往訪閲覧,2-3①,3-3,要,...,閲覧縦覧ー共通１,告示、通知・通達等の発出又は改正,325AC0000000239,商品先物取引法,昭和二十五年法律第二百三十九号,None,302,3,None,NaN
7529,新規,437,商品投資に係る事業の規制に関する法律,経済産業省\n農林水産省,第24条第1項,商品投資顧問契約に係る顧客の財産に関する帳簿書類の閲覧,往訪閲覧,2-3①,3-3,要,...,閲覧縦覧ー共通１,告示、通知・通達等の発出又は改正,403AC0000000066,商品投資に係る事業の規制に関する法律,平成三年法律第六十六号,None,24,1,None,NaN
7530,新規,438,海洋汚染等及び海上災害の防止に関する法律,国土交通省,第９条の14,財務諸表等の閲覧等,往訪閲覧,2-3①\n2-3②\n2-3③,3-3,要,...,閲覧縦覧ー共通５,告示、通知・通達等の発出又は改正,345AC0000000136,海洋汚染等及び海上災害の防止に関する法律,昭和四十五年法律第百三十六号,None,9_14,None,None,NaN


In [8]:
df_with_clause_num.to_json(
    (data_dir / "reg_list.json"),
    orient="split",
    force_ascii=False,
    index=False,
)

In [9]:
sched_wb = xl.load_workbook(orig_xlsx)
sched_sheet = sched_wb["工程表の類型"]

def get_sched_head(sheet: Worksheet):
    ret = []
    yr = ""
    for c in range(2, 5 + 1):
        yr = sheet.cell(2, c).value or yr
        mo = sheet.cell(3, c).value
        ret.append((c, yr, mo))
    return ret

sched_head = get_sched_head(sched_sheet)
display(sched_head)


工程表一覧表
工程表の類型


[(2, '令和４年度', '１月～３月'),
 (3, '令和５年度', '４月～９月'),
 (4, '令和５年度', '10月～３月'),
 (5, '令和６年度', '４月～６月')]

In [10]:
def get_sched_row_groups(sheet: Worksheet, min_row = 4, max_row = 10000):
    row_groups: List[Tuple[str, int, int]] = [] # half-open
    max_row = min(max_row, sheet.max_row)
    rstart = min_row
    name = None
    for r in range(min_row, max_row + 1):
        all_blank = all(not sheet.cell(r, c).value for c in range(1, 5 + 1))
        new_name = sheet.cell(r, 1).value

        if name is None:
            name = new_name

        if (new_name and (name != new_name)) or all_blank or (r == max_row):
            row_groups.append((name, rstart, r + 1 if (r == max_row) else r))
            rstart = r
            name = new_name

        if all_blank:
            break

    return row_groups

sched_row_groups = get_sched_row_groups(sched_sheet)

In [11]:
def get_sched_items(sheet: Worksheet, row_groups: List[Tuple[str, int, int]], min_col=2, max_col=10):
    max_col = min(max_col, sheet.max_column)
    groups = []

    for group_name, rstart, rend in row_groups:

        items: List[Tuple[str, int, int]] = [] # half-open
        for r in range(rstart, rend):
            cstart = min_col
            name = None
            for c in range(min_col, max_col + 1):
                cell = sheet.cell(r, c)
                merged = isinstance(cell, MergedCell)

                new_name = sheet.cell(r, c).value

                if name is None:
                    cstart = c
                    name = new_name

                if (name != new_name) and (not merged) and ((not new_name) or (c == max_col)):
                    if name:
                        items.append((name, cstart, c + 1 if (c == max_col) else c))
                    cstart = c
                    name = new_name

        groups.append((group_name, items))
    return groups

sched_items = get_sched_items(sched_sheet, sched_row_groups)
display(sched_items)

[('目視ー共通１', [('法令等改正手続', 2, 3)]),
 ('目視ー共通２', [('実態把握（各省自ら実施）', 2, 3), ('法令等改正手続', 3, 4)]),
 ('目視ー共通３', [('実態把握（各省自ら実施）', 2, 3), ('対外調整等', 3, 4), ('法令等改正手続', 3, 5)]),
 ('目視ー共通４', [('実態把握（外部委託調査等）', 2, 4), ('法令等改正手続', 4, 5)]),
 ('目視ー共通５', [('実態把握（外部委託調査等）', 2, 4), ('対外調整等', 4, 5), ('法令等改正手続', 4, 6)]),
 ('目視ー共通６', [('実態把握（技術検証等）', 2, 4), ('法令等改正手続', 4, 5)]),
 ('目視ー共通７', [('実態把握（技術検証等）', 2, 5), ('法令等改正手続', 4, 6)]),
 ('目視ー共通８', [('実態把握（技術検証等）', 2, 4), ('対外調整等', 4, 5), ('法令等改正手続', 4, 6)]),
 ('目視ー共通９', [('実態把握（技術検証等）', 2, 5), ('対外調整等', 4, 5), ('法令等改正手続', 4, 6)]),
 ('目視ー内閣府１', [('実態把握（各省自ら実施）', 2, 5), ('対外調整等', 3, 5), ('法令等改正手続', 4, 5)]),
 ('目視ー警察庁１', [('実態把握（技術検証等）', 2, 6), ('対外調整等', 4, 6), ('法令等改正手続', 4, 6)]),
 ('目視ー総務省１', [('実態把握（各省自ら実施）', 2, 5), ('法令等改正手続', 4, 6)]),
 ('目視ー法務省１', [('実態把握（技術検証等）', 2, 5), ('対外調整等', 5, 6), ('法令等改正手続', 5, 6)]),
 ('目視ー法務省２', [('実態把握（各省自ら実施）', 2, 4), ('法令等改正手続', 4, 5)]),
 ('目視ー法務省３', [('法令等改正手続', 2, 4)]),
 ('目視ー厚生労働省１', [('実態把握（各省自ら実施）', 2, 4), ('対外調整等', 2, 5), 

In [12]:
with open(data_dir / "sched.json", "w", encoding="utf-8") as f:
    json.dump(
        {"head": sched_head, "items": sched_items},
        f,
        ensure_ascii=False,
    )